In [1]:
import pandas as pd
import numpy as np
import requests
import ast
import re

In [2]:
resale_data = pd.read_csv('resale_data.csv',parse_dates=['month'],dayfirst=True)

Using the base dataset (resale_data) already compiled, we want to query new data from the data.gov.sg API and append it to the current dataset

### Input requested date range below

In [3]:
#Get data in daterange (left and right-inclusive), in format "YYYY-MM-01"
date1 = '2021-09-01'
date2 = '2021-12-01'

In [4]:
#URL for the API
datagov_api = "https://data.gov.sg/api/action/datastore_search?resource_id=f1765b54-a209-4718-8d38-a39237f502b3&limit=400000"

#Query the API for the latest dataset
new_resale_data = pd.DataFrame(requests.get(datagov_api).json()['result']['records'])

#Convert string to month
new_resale_data['month'] = pd.to_datetime(new_resale_data['month'])

#Convert resale price and floor area to int
new_resale_data['resale_price'] = new_resale_data['resale_price'].astype(float).round(0).astype(int)
new_resale_data['floor_area_sqm'] = new_resale_data['floor_area_sqm'].astype(float)

#Convert floor area from sq metres to sq foot (more commonly used)
new_resale_data['floor_area_sqft'] = new_resale_data['floor_area_sqm'].apply(lambda x: x*10.7639).round(2)
new_resale_data.drop('floor_area_sqm',axis=1,inplace=True)

new_resale_data = new_resale_data[new_resale_data['month'].between(date1,date2,inclusive='both')].reset_index(drop=True)

The following is a simple find and replace function. This part requires some data exploration, as some locations in the data.gov.sg and OneMap APIs do not have the same spelling (e.g. Data.gov spells "ST. GEORGE'S LANE", OneMap spells "SAINT GEORGE'S LANE")

In [5]:
def find_replace(x):
    substr = x.replace("ST. GEORGE'S","SAINT GEORGE'S")
    return substr

new_resale_data['street_name'] = new_resale_data['street_name'].apply(find_replace)

#Get the full address from combining the block number and street name
new_resale_data['address'] = new_resale_data['block']+" "+new_resale_data['street_name']

The following dataset (address_info.csv) is an extract from the OneMap API, which provides us with the postal code, building name (if applicable) and coordinates of an address. As querying the API directly takes a long time (server limits to 250 requests per minute), you can use the dataset provided to get location data by address.

In [6]:
address_info = pd.read_csv('address_info.csv')

#Pandas reads the postal code as an integer instead of a string, so postal codes starting with '0' will lose it
#Convert the postcode to a string and pad the front with zeros
address_info['postcode'] = address_info['postcode'].astype(str).str.zfill(6)

#Convert the string form of the coordinates to a tuple
address_info['coordinates'] = address_info['coordinates'].apply(ast.literal_eval)

#Convert the dataframe into a dictionary
#The format of the dictionary is {address: [postcode, building_name, coordinates]}
address_dict = address_info.set_index('address').T.to_dict('list')

#Map the values onto the dataset
for n,field in enumerate(['postcode','building']):
    new_resale_data[field] = new_resale_data['address'].map(lambda x: address_dict[x][n] if x in address_dict.keys() else np.nan)

If the address is not found in the address_dict, Pandas will return NaN values for that row.

In [7]:
def split_lease(x):
    """
    Convert the lease remaining from "{x} years {y} months" into a float
    """
    try:
        return float(x)
    except:
        years = re.findall(r'\d+(?= years)',x)
        months = re.findall(r'\d+(?= months)',x)
        years,months = map(lambda a: int(a[0]) if len(a)>0 else 0,[years,months])
        remaining = years+(months/12)

        return round(remaining,2)
    
#Convert remaining lease to years
new_resale_data['remaining_lease'] = new_resale_data['remaining_lease'].apply(split_lease)

#Add resale price per square metre
new_resale_data['psf'] = new_resale_data['resale_price']/new_resale_data['floor_area_sqft']

#Rename columns
new_resale_data.rename(columns={'street_name':'street'},inplace=True)

In [8]:
#Check if there are any missing values
new_resale_data.isnull().sum()

town                   0
flat_type              0
flat_model             0
street                 0
resale_price           0
month                  0
remaining_lease        0
lease_commence_date    0
storey_range           0
_id                    0
block                  0
floor_area_sqft        0
address                0
postcode               0
building               0
psf                    0
dtype: int64

The following code will query the OneMap REST API with the given address (block + street) of the entries with missing values. It will then extract the postcode and building name.

If the search query returns more than 1 result, it will print the building names of the results and user will be prompted to choose the correct building.

The null values of the new resale data are then filled in, and the address_info will be updated.

In [9]:
# Add your OneMap private token here
# For instructions on how to get your token, visit https://discuss.onemap.sg/t/steps-for-api-authentication/59
token = None

class OneMap:
    def __init__(self,token):
        self.token = token
        self.common = "https://developers.onemap.sg/commonapi"
        self.private = "https://developers.onemap.sg/privateapi"
    def rest(self,searchVal,returnGeom='Y',getAddrDetails='Y',pageNum=None):
        url = self.common+f"/search?searchVal={searchVal}&returnGeom={returnGeom}&getAddrDetails={getAddrDetails}"
        if pageNum is not None:
            try:
                url = url+f"&pageNum={pageNum}"
            except:
                pass
        return requests.get(url).json()
    
    def routing(self):
        """
        method still in progress
        """
        pass
    
onemap = OneMap(token)

In [10]:
for i,row in new_resale_data[new_resale_data['postcode'].isnull()].iterrows():
    omresult = onemap.rest(row['address'])
    if omresult['found']==1:
        j = 0
    else:
        results = []
        for result in omresult['results']:
            results.append(result['BUILDING'])
        print(f"address_info is: {row['address']}")
        for n,r in enumerate(results):
            print(f'Result {n}: {r}')
        j = int(input('\nWhich result is correct? '))
    
    postcode = omresult['results'][j]['POSTAL']
    building = omresult['results'][j]['BUILDING']
    lat = omresult['results'][j]['LATITUDE']
    lng = omresult['results'][j]['LONGITUDE']
        
    building = row['address'] if building=='NIL' else building
    
    address_info = address_info.append({'postcode':postcode,
                                       'address_info':row['address'],
                                       'building':building,
                                       'block':row['block'],
                                       'street':row['street'],
                                       'coordinates':(lat,lng)},ignore_index=True)
    
    new_resale_data.loc[i,'postcode'] = postcode
    new_resale_data.loc[i,'building'] = building

In [11]:
#Concat the old and new dataset and drop irrelevant columns
final_resale_data = pd.concat([resale_data,new_resale_data],ignore_index=True).drop(['_id','address'],axis=1)

#Sort the address_info dataframe
address_info = address_info.sort_values(['postcode','address']).reset_index(drop=True)

In [12]:
#Export as csv
# final_resale_data.to_csv('new_resale_data.csv')
# address_info.to_csv('new_address_info.csv')